# ID3 algorithm of Decision Tree

ID3 stands for Iterative Dichotomiser 3 and is named such because the algorithm iteratively (repeatedly) dichotomizes(divides) features into two or more groups at each step.

ID3 uses Information Gain or just Gain to find the best feature.


<h3>ID3 Steps

<h6>Calculate the Information Gain of each feature.
<h6>Considering that all rows don’t belong to the same class, split the dataset S into subsets using the feature for which the Information Gain is maximum.
<h6>Make a decision tree node using the feature with the maximum Information gain.
<h6>If all rows belong to the same class, make the current node as a leaf node with the class as its label.
<h6>Repeat for the remaining features until we run out of all features, or the decision tree has all leaf nodes.

In [1]:
import pandas as pd
import math
import numpy as np


data=pd.read_csv("dataset_tennis.csv")
print(data)
features = [feat for feat in data]
features.remove("answer")



     Outlook Temperature Humidity    Wind answer
0      sunny         hot     high    weak     no
1      sunny         hot     high  strong     no
2   overcast         hot     high    weak    yes
3       rain        mild     high    weak    yes
4       rain        cool   normal    weak    yes
5       rain        cool   normal  strong     no
6   overcast        cool   normal  strong    yes
7      sunny        mild     high    weak     no
8      sunny        cool   normal    weak    yes
9       rain        mild   normal    weak    yes
10     sunny        mild   normal  strong    yes
11  overcast        mild     high  strong    yes
12  overcast         hot   normal    weak    yes
13      rain        mild     high  strong     no


In [2]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""



In [3]:
def entropy(examples):
    positive = 0.0
    negative = 0.0
    for _, row in examples.iterrows():
        if row["answer"] == "yes":
            positive += 1
        else:
            negative += 1
    if positive == 0.0 or negative == 0.0:
        return 0.0
    else:
        p = positive / (positive + negative)
        n = negative / (positive + negative)
        return -(p * math.log(p, 2) + n * math.log(n, 2))  #formula to calculate Entropy



In [4]:
def gain_calc(examples, attr):  #Function to Calculate Gain for a given attribute
    uniq = np.unique(examples[attr])
   
    gain = entropy(examples)
 
    for u in uniq:
        subdata = examples[examples[attr] == u]
      
        sub_e = entropy(subdata)
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
       
    return gain



In [5]:
def ID3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
      
        gain = gain_calc(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
    
    uniq = np.unique(examples[max_feat])
   
    for u in uniq:
        
        subdata = examples[examples[max_feat] == u]
       
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["answer"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)
    return root



In [6]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value,"", end="")
    if root.isLeaf:
        print("-> ", root.pred)
    print()

    for child in root.children:
        printTree(child, depth + 1)

root = ID3(data, features)
printTree(root)


Outlook 
	overcast ->  ['yes']

	rain 
		Wind 
			strong ->  ['no']

			weak ->  ['yes']

	sunny 
		Humidity 
			high ->  ['no']

			normal ->  ['yes']

